In [1]:
!CXXFLAGS+=-stdlib=libc++ maturin develop --release

🔗 Found pyo3 bindings
🐍 Found CPython 3.11 at /Users/josh/josh/projects/tech/flow/.venv/bin/python
💻 Using `MACOSX_DEPLOYMENT_TARGET=10.7` for x86_64-apple-darwin by default
   Compiling flow v0.1.0 (/Users/josh/josh/projects/tech/flow)
    Finished release [optimized + debuginfo] target(s) in 13.08s              
📦 Built wheel for CPython 3.11 to /var/folders/w9/c8qym84j2pg4fjvx_bzr84bc0000gn/T/.tmp91fT7W/flow-0.1.0-cp311-cp311-macosx_10_7_x86_64.whl
🛠 Installed flow-0.1.0


In [4]:
import traceback

import contextplot
import matplotlib
import matplotlib.collections
import matplotlib.patches
import numpy as np
import tqdm

import flow

contextplot.set_defaults()

NS_PER_S = 1e9

num_cells = 100
num_z_cells = 5
kinematic_viscosity = 0.01

# example = 'bump_1d'
# amplitude = 0.2
# width = 0.2
# t_max = 5
# solver = getattr(flow, example)(num_cells, num_z_cells, kinematic_viscosity, amplitude, width)

example = 'hill_1d'
height = 0.4
t_max = 5
solver = getattr(flow, example)(num_cells, num_z_cells, kinematic_viscosity, height)

x_vertices = solver.grid.x_axis.vertices
x_centers = solver.grid.x_axis.centers

dt_max = 0.01
frame_rate_hz = 30
t_max_ns = int(t_max * NS_PER_S)
speedup_factor = 0.5

courant_margin = 20
dt_plot_ns = int(1 / frame_rate_hz * speedup_factor * NS_PER_S)

def frame_index(t_ns: int):
    return t_ns // dt_plot_ns

with contextplot.context_video(f'videos/{example}.mp4', 1, size_inches=(6.5, 3), dpi=300, frame_rate_hz=frame_rate_hz) as cv:
    try:
        t_ns = 0
        bar = tqdm.tqdm(total=t_max_ns)
        should_plot = True
        while t_ns < t_max_ns:
            if should_plot:
                with cv.next_frame() as cp:
                    z_lattice = solver.z_lattice
                    pressure = solver.pressure
                    velocity = solver.velocity

                    poly_verts = []
                    pressures = []
                    for i in range(z_lattice.shape[0] - 1):
                        for j in range(z_lattice.shape[2] - 1):
                            poly_verts.append([
                                [x_vertices[i], z_lattice[i, 0, j]],
                                [x_vertices[i + 1], z_lattice[i + 1, 0, j]],
                                [x_vertices[i + 1], z_lattice[i + 1, 0, j + 1]],
                                [x_vertices[i], z_lattice[i, 0, j + 1]],
                            ])
                            pressures.append(pressure[i, 0, j, 1])
                    poly_collection = matplotlib.collections.PolyCollection(
                        poly_verts,
                        closed=True,
                        array=pressures,
                        cmap=matplotlib.cm.viridis,
                        edgecolors='black',
                        linewidth=0.,
                        snap=True,
                        norm=matplotlib.colors.Normalize(vmin=0, vmax=5),
                    )
                    outs = cp.ax_raw.add_collection(poly_collection)
                    _ = cp.figure.colorbar(outs, label='Kinematic pressure')

                    cp.ax.quiver(
                        np.broadcast_to(x_centers[:, np.newaxis], z_lattice[:-1, 0, :-1].shape),
                        0.25 * (z_lattice[:-1, 0, :-1] + z_lattice[1:, 0, :-1] + z_lattice[:-1, 0, 1:] + z_lattice[1:, 0, 1:]),
                        velocity[:, 0, :, 0, 0],
                        velocity[:, 0, :, 0, 2],
                        pivot='tail',
                        scale=10,  # Bigger means shorter arrows.
                        scale_units='inches',
                        alpha=0.6,
                    )

                    cp.ax.set(
                        xlim=[solver.grid.x_axis.vertices[0], solver.grid.x_axis.vertices[-1]],
                        ylim=[0, 1.5],
                        aspect='equal',
                        title=f'{example}, t = {t_ns / NS_PER_S:.3f} s',
                    )

            courant_dt = solver.courant_dt
            dt_ns = int(min(dt_max, courant_dt.min() / courant_margin) * NS_PER_S)
            if dt_ns == 0:
                index = np.unravel_index(courant_dt.argmin(), courant_dt.shape)
                x = solver.grid.x_axis.centers[index[0]]
                y = solver.grid.y_axis.centers[index[1]]
                z = solver.z_lattice[index[0], index[1], index[2]]
                v = solver.velocity[*index]
                raise ValueError(f"CFL is approaching zero at x={x} y={y} z={z} where v={v}")

            current_frame_index = frame_index(t_ns=t_ns)
            future_frame_index = frame_index(t_ns=t_ns + dt_ns)
            if current_frame_index != future_frame_index:
                dt_ns = (current_frame_index + 1) * dt_plot_ns - t_ns
                should_plot = True
            else:
                should_plot = False
            
            bar.set_description(f'dt: {dt_ns / NS_PER_S:.9f}')

            solver.step(dt_ns / NS_PER_S)
            t_ns += dt_ns
            if t_ns > t_max_ns:
                break
            _ = bar.update(dt_ns)
    except KeyboardInterrupt:
        pass
    except BaseException:
        traceback.print_exc()
    finally:
        bar.close()

dt: 0.000000001:  72%|███████▏  | 3612698928/5000000000 [01:53<00:50, 27244595.95it/s]Traceback (most recent call last):
  File "/var/folders/w9/c8qym84j2pg4fjvx_bzr84bc0000gn/T/ipykernel_38865/4063807634.py", line 107, in <module>
    raise ValueError(f"CFL is approaching zero at x={x} y={y} z={z} where v={v}")
ValueError: CFL is approaching zero at x=2.205 y=0.005 z=0.7125280820882652 where v=[-4.18968138e+05  4.85736457e-09 -4.76548011e+05]
dt: 0.000000001:  72%|███████▏  | 3612698929/5000000000 [01:53<00:43, 31935480.20it/s]
